In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import csv
import json
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime, timezone

In [2]:
# Load .env variables
load_dotenv()

True

## Fetching Stock Data

In [3]:
# Set Alpaca API key and secret key
alpaca_key = os.getenv('ALPACA_API_KEY')
alpaca_priv_key = os.getenv('ALPACA_SECRET_KEY')

In [4]:
# Check data types to ensure key imports were successful
print(type(alpaca_key))
print(type(alpaca_priv_key))

<class 'str'>
<class 'str'>


In [5]:
# Create alpaca API object
api = tradeapi.REST(
    alpaca_key,
    alpaca_priv_key,
    api_version = 'v2')

In [6]:
# Create parameters to be used in data pulling
start = pd.Timestamp('2017-05-05', tz='America/New_York').isoformat()
end = pd.Timestamp('2022-05-05', tz='America/New_York').isoformat()
tickers = ['AAPL', 'MSFT', 'AMZN' , 'TSLA', 'GOOGL']
timeframe = '1Day'

In [7]:
# Pull data
stock_pulled_df = api.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

In [8]:
# Slice dataframe
aapl = stock_pulled_df[stock_pulled_df['symbol']=='AAPL'].drop('symbol', axis=1)
msft = stock_pulled_df[stock_pulled_df['symbol']=='MSFT'].drop('symbol', axis=1)
amzn = stock_pulled_df[stock_pulled_df['symbol']=='AMZN'].drop('symbol', axis=1)
tsla = stock_pulled_df[stock_pulled_df['symbol']=='TSLA'].drop('symbol', axis=1)
googl = stock_pulled_df[stock_pulled_df['symbol']=='GOOGL'].drop('symbol', axis=1)

# Concatenate sliced data into new dataframe
stock_df = pd.concat(
    [aapl, msft, amzn, tsla, googl],
    axis=1,
    keys=['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL']
)

In [9]:
# Index dataframe by date
stock_df.index=stock_df.index.date

In [10]:
# Drop any Null values
stock_df.dropna()

# Remove vwap column
stock_df['AAPL'].drop('vwap', axis=1)

,open,high,low,close,volume,trade_count
2017-05-05,146.76,148.98,146.76,148.96,27359822,157911
2017-05-08,149.03,153.70,149.03,153.01,48864610,255930
2017-05-09,153.88,154.88,153.45,153.99,39135644,221722
2017-05-10,153.63,153.94,152.11,153.26,25807492,156839
2017-05-11,152.69,154.07,152.31,153.95,27259011,137633
...,...,...,...,...,...,...
2022-04-29,161.84,166.20,157.25,157.65,131724463,1028256
2022-05-02,156.65,158.23,153.27,157.96,122860515,1148543
2022-05-03,158.06,160.71,156.32,159.48,88581314,702272
2022-05-04,159.65,166.48,159.26,166.02,108376463,867145


In [24]:
# Save data to csv
stock_df.to_csv('stock_data.csv')

## Fetching Crypto Data

In [12]:
# Defining URLs
btc_url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1493906401&to=1651759199'
eth_url = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range?vs_currency=usd&from=1493906401&to=1651759199'
luna_url = 'https://api.coingecko.com/api/v3/coins/terra-luna/market_chart/range?vs_currency=usd&from=1493906401&to=1651759199'
sol_url = 'https://api.coingecko.com/api/v3/coins/solana/market_chart/range?vs_currency=usd&from=1493906401&to=1651759199'
ada_url = 'https://api.coingecko.com/api/v3/coins/cardano/market_chart/range?vs_currency=usd&from=1493906401&to=1651759199'

In [13]:
# Perform GET request to pull data for cryptos
btc_request = requests.get(btc_url)
eth_request = requests.get(eth_url)
luna_request = requests.get(luna_url)
sol_request = requests.get(sol_url)
ada_request = requests.get(ada_url)

In [14]:
# Convert pulled data into json
btc_request = btc_request.json()
eth_request = eth_request.json()
luna_request = luna_request.json()
sol_request = sol_request.json()
ada_request = ada_request.json()

In [15]:
# Create and organize dataframes for cryptos
btc_df = pd.DataFrame(btc_request['prices'])
eth_df = pd.DataFrame(eth_request['prices'])
luna_df = pd.DataFrame(luna_request['prices'])
sol_df = pd.DataFrame(sol_request['prices'])
ada_df = pd.DataFrame(ada_request['prices'])

In [16]:
# Rename columns
btc_df.rename(columns = {0 : 'Date', 1 : 'Price'}, inplace=True)
eth_df.rename(columns = {0 : 'Date', 1 : 'Price'}, inplace=True)
luna_df.rename(columns = {0 : 'Date', 1 : 'Price'}, inplace=True)
sol_df.rename(columns = {0 : 'Date', 1 : 'Price'}, inplace=True)
ada_df.rename(columns = {0 : 'Date', 1 : 'Price'}, inplace=True)

In [17]:
# Convert unix time format into traditonal date format
btc_df['date'] = pd.to_datetime(btc_df['Date'], unit='ms')
eth_df['date'] = pd.to_datetime(eth_df['Date'], unit='ms')
luna_df['date'] = pd.to_datetime(luna_df['Date'], unit='ms')
sol_df['date'] = pd.to_datetime(sol_df['Date'], unit='ms')
ada_df['date'] = pd.to_datetime(ada_df['Date'], unit='ms')

In [18]:
# Set new date format as index
btc_df.set_index('date', inplace=True)
eth_df.set_index('date', inplace=True)
luna_df.set_index('date', inplace=True)
sol_df.set_index('date', inplace=True)
ada_df.set_index('date', inplace=True)

In [19]:
# Drop unix column
btc_df.drop(['Date'], axis=1, inplace=True)
eth_df.drop(['Date'], axis=1, inplace=True)
luna_df.drop(['Date'], axis=1, inplace=True)
sol_df.drop(['Date'], axis=1, inplace=True)
ada_df.drop(['Date'], axis=1, inplace=True)

In [20]:
# Concatenate crypto dataframes
crypto_df = pd.concat(
    [btc_df, eth_df, luna_df, sol_df, ada_df],
    axis=1,
    keys=['BTC', 'ETH', 'LUNA', 'SOL', 'ADA']
)

#### Possibly change cryptos to ones that are high in market cap ranking but been around for 5 years

#### e.g btc, eth, xrp

In [21]:
crypto_df

,BTC,ETH,LUNA,SOL,ADA
,Price,Price,Price,Price,Price
date,,,,,
2017-05-05,1506.043800,91.160387,NaN,NaN,NaN
2017-05-06,1547.815000,96.427680,NaN,NaN,NaN
2017-05-07,1557.883800,89.739679,NaN,NaN,NaN
2017-05-08,1633.380000,87.948120,NaN,NaN,NaN
2017-05-09,1700.177500,87.502444,NaN,NaN,NaN
...,...,...,...,...,...
2022-05-01,37820.611765,2738.174136,78.702627,85.302984,0.758064
2022-05-02,38537.654767,2832.513107,82.316205,89.738346,0.790253


## Stock Price 5 Years Ago

In [22]:
# Stock price from 5 years ago
aapl_open = stock_df['AAPL']['open']

aapl_open_2017_05_05 = aapl_open.iloc[0:10]

In [23]:
aapl_open_2017_05_05

2017-05-05    146.76
2017-05-08    149.03
2017-05-09    153.88
2017-05-10    153.63
2017-05-11    152.69
2017-05-12    155.00
2017-05-15    156.01
2017-05-16    156.00
2017-05-17    153.94
2017-05-18    151.25
Name: open, dtype: float64